# An Introduction to Value Function Iteration (Part 2)

Quentin Batista, The University of Tokyo

## Overview

1. [The Bellman Operator and the Value Function Iteration (VFI) algorithm](#The-Bellman-Operator)
2. [The approximate Bellman Operator and the Fitted Value Function Iteration (FVFI) algorithm](#The-Approximate-Bellman-Operator)
3. [Solving the cake eating problem with FVFI in Python](#Python-Implementation)
4. [The Curse of Dimensionality](#The-Curse-of-Dimensionality)
5. [Brief, high-level discussion of variants on the basic algorithm](#Variants-on-the-Basic-Algorithm)
6. [A Simple Stochastic DP Example](#Stochastic-DP)

## Python Implementation

### Some comments on programming language

Many people have strong opinions about which language is better.

**Advantages of Matlab:**
- Currently mainstream in Economics (legacy code is often in Matlab)

**Disadvantages of Matlab:**
- Closed-source
- Often slow
- Design

**Advantages of Python:**
- Well-developed ecosystem
- Extremely versatile
- Object-orientated language

**Disadvantages of Python:**
- Getting different tools to work together can be difficult

**Advantages of Julia:**
- Built for scientific computation
- Dynare developers are working on a Julia version

**Disadvantages of Julia:**
- Changing very quickly


Some people argue that Python is too slow for computational Economics (e.g. [Aruoba and Fernandez-Villaverde](https://github.com/jstac/julia_python_comparison/blob/master/Update_March_23_2018.pdf))

Answer: [No, Python is Not Too Slow for Computational Economics](https://notes.quantecon.org/submission/5bae5cb538674f000fd2c8e3) by John Stachurski

### Functional vs non-functional requirements 

In software engineering a functional requirement specifies what the system should do/how it should look, etc.

Non-functional requirements specify how the system should be. For example:

- Extensibility
- Reliability   
- Accessibility  
- Maintainability  
- Testability  
- Scalability
- etc.

### Compiled vs. interpreted languages

**Interpreted:** Python, Javascript, Matlab, ...  
**Compiled:** C++, Fortran, Julia (just-in-time), ...

Workflow with compiled languages:
1. Write your program
2. Use a **compiler** to translate your program into machine readable code
3. Run your program

Workflow with interpreted languages:
1. Write your program
2. Run your program

With interpreted languages, each line is read and executed **at runtime**. The interpreter is the program that takes care of getting your computer to run your program.

Example:

```python
x = 1.
y = 2.
x + y
```

Roughly speaking, this provides a much more dynamic environment but execution is relatively slower.

**Tradeoff:** Developer productivity vs. execution time

**Note 1:** This distinction is relatively arbitrary because the interpreter might act just like a compiler

**Note 2:** Python now has a just-in-time compiler called Numba for numerical computation. This tool comes with additional restrictions on what you can do. We make extensive use of this tool in QuantEcon lectures. 

The general idea is that in Python you want to figure out which parts of your code are slow and optimize only those parts: "Premature optimization is the root of all evil" -- Donald Knuth 

### Python tools for scientific computation

- [NumPy](https://numpy.org/devdocs/reference/index.html): Data structure for numerical computation + some linear algebra
- [SciPy](https://docs.scipy.org/doc/scipy/reference/): Scientific computation from solving optimization problem, linear algebra, interpolation, solving differential equations, etc...
- [Pandas](https://pandas.pydata.org/): Data structure + tools for data analysis
- [Matplotlib](https://matplotlib.org/): Plotting library
- [SymPy](https://www.sympy.org/en/index.html): Symbolic computation
- [Numba](http://numba.pydata.org/): Makes your code fast
- [Interpolation](https://github.com/EconForge/interpolation.py): Fast interpolation 
- [JAX](https://github.com/google/jax): Automatic differentiation + code optimization

### Some Python tools for economic modelling

- [QuantEcon.py](https://quanteconpy.readthedocs.io/en/latest/): Tools to support your own code (e.g. discrete DP, discretization of AR(1) process, LQ control problems, some optimizers, etc...).
- [Dolo](https://dolo.readthedocs.io/en/latest/): More or less equivalent to `Dynare`.
- [Econ-ARK](https://econ-ark.org/): Toolkit for heterogenuous agents structural modeling.

If you **really** love neural networks: [DeepEquilibriumNets](https://github.com/sischei/DeepEquilibriumNets)

In [ ]:
!pip install interpolation matplotlib quantecon numba==0.48.0

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from interpolation import interp
from scipy.optimize import minimize_scalar, minimize, curve_fit
from numba import njit, prange 
from quantecon.optimize import scalar_maximization
from interpolation.splines import eval_linear, UCGrid


%config InlineBackend.figure_format = 'retina'

In [ ]:
# "Mathematics is the art of giving the same thing different names" (Henri Poincaré)

class CakeEating:
    def __init__(self,
                 β=0.96,           # discount factor
                 γ=1.5,            # degree of relative risk aversion
                 x_grid_min=1e-3,  # exclude zero for numerical stability
                 x_grid_max=2.5,   # size of cake
                 x_grid_size=120):

        self.β, self.γ = β, γ

        # Set up grid
        self.x_grid = np.linspace(x_grid_min, x_grid_max, x_grid_size)

    # Utility function
    def u(self, c):

        γ = self.γ

        if γ == 1:
            return np.log(c)
        else:
            return (c ** (1 - γ)) / (1 - γ)

    # first derivative of utility function
    def u_prime(self, c):

        return c ** (-self.γ)

    def state_action_value(self, c, x, v_array):
        """
        Right hand side of the Bellman equation given x and c.
        
        """

        u, β = self.u, self.β
        v = lambda x: interp(self.x_grid, v_array, x)

        return u(c) + β * v(x - c)


In [ ]:
# Create an instance of the `CakeEating` class
# ce.β
ce = CakeEating()

In [ ]:
# Define a function to solve the maximization problem
def maximize(g, a, b, args):
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum


def T(v, ce):
    """
    The Bellman operator.  Updates the guess of the value function.

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)

    for i, x in enumerate(ce.x_grid):
        # Maximize RHS of Bellman equation at state x
        v_new[i] = maximize(ce.state_action_value, 1e-10, x, (x, v))[1]  # v_new[i] = v'(x_i)

    return v_new

In [ ]:
# This cell creates a plot showing how the value function changes at each iteration
x_grid = ce.x_grid
v = ce.u(x_grid)       # Initial guess
n = 12                 # Number of iterations

fig, ax = plt.subplots()

ax.plot(x_grid, v, color=plt.cm.jet(0),
        lw=2, alpha=0.6, label='Initial guess')

for i in range(n):
    v = T(v, ce)  # Apply the Bellman operator
    ax.plot(x_grid, v, color=plt.cm.jet(i / n), lw=2, alpha=0.6)

ax.legend()
ax.set_ylabel('value', fontsize=12)
ax.set_xlabel('cake size $x$', fontsize=12)
ax.set_title('Value function iterations')

plt.show()

In [ ]:
def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1

    while i < max_iter and error > tol:  # `max_iter` is used to ensure termination of algorithm
        v_new = T(v, ce)

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new

In [ ]:
%%time

v = compute_value_function(ce)

In [ ]:
# Compute analytical solution at grid points
def v_star(x, β, γ):
    return (1 - β**(1 / γ))** (-γ) * (x ** (1-γ) / (1-γ))


v_analytical = v_star(ce.x_grid, ce.β, ce.γ)

In [ ]:
# This cell creates a plot that compares the numerical and analytical solutions
fig, ax = plt.subplots()

ax.plot(x_grid, v_analytical, label='analytical solution')
ax.plot(x_grid, v, label='numerical solution')
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
ax.set_title('Comparison between analytical and numerical value functions')
plt.show()

**What's going on near 0?**

$$\begin{eqnarray}
Lv^{*}\left(x_{0}\right)&\approx&\max_{0\leq c\leq x_{0}}u\left(c\right)+\beta Lv^{*}\left(x_{0}-c\right)\\&=&\max_{0\leq c\leq x_{0}}u\left(c\right)+\beta Lv^{*}\left(x_{0}\right)\\&=&u\left(x_{0}\right)+\beta u\left(x_{0}\right)+\beta^{2}u\left(x_{0}\right)+\dots\\&=&\frac{u\left(x_{0}\right)}{1-\beta}
\end{eqnarray}$$

In [ ]:
ce.u(x_grid.min()) / (1 - ce.β)

In [ ]:
v[0]

In [ ]:
ce.u(x_grid.min() * 0.99) / (1 - ce.β)

In [ ]:
def σ(ce, v):
    """
    The optimal policy function. Given the value function,
    it finds optimal consumption in each state.

    * ce is an instance of CakeEating
    * v is a value function array

    """
    c = np.empty_like(v)

    for i in range(len(ce.x_grid)):
        x = ce.x_grid[i]
        # Maximize RHS of Bellman equation at state x
        c[i] = maximize(ce.state_action_value, 1e-10, x, (x, v))[0]

    return c


c = σ(ce, v)

In [ ]:
def c_star(x, β, γ):

    return (1 - β ** (1/γ)) * x


c_analytical = c_star(ce.x_grid, ce.β, ce.γ)

In [ ]:
fig, ax = plt.subplots()

ax.plot(ce.x_grid, c_analytical, label='analytical')
ax.plot(ce.x_grid, c, label='Numerical')
ax.set_ylabel(r'$\sigma(x)$')
ax.set_xlabel('$x$')
ax.legend()

plt.show()

## The Curse of Dimensionality

We ask the following question: how does the number of points in our grid change when we increase the number of dimensions of the state vector?

In [ ]:
dims_nb = 12
d = np.arange(1, dims_nb)
points_per_d = 10
grid_points = points_per_d ** d

In [ ]:
plt.plot(d, grid_points, 'o');
plt.xlabel('number of dimensions')
plt.ylabel('total numer of grid points')
# 1e11 = one hundred billion

**Both memory requirements and the amount of computation explode exponentially as the number of dimensions grow!**

Many economic models feature high-dimensional state spaces:
- International Real Business Cycle models
- Dynamic Stochastic General Equilibrium (DSGE) models
- Overlapping Generations (OLG) models
- Mathematical finance

### CPU-bound

"In computer science, a computer is CPU-bound (or compute-bound) when the time for it to complete a task is determined principally by the speed of the central processor: processor utilization is high, perhaps at 100% usage for many seconds or minutes. " (Wikipedia)

### Memory-bound

"Memory bound refers to a situation in which the time to complete a given computational problem is decided primarily by the amount of memory required to hold data." (Wikipedia)

In [ ]:
dims_nb = 15

try:
    np.ones((points_per_d, ) * dims_nb)
except MemoryError as e: 
    print('Error message:', e)

## Variants on the Basic Algorithm

### Different Numerical Grids

**Basic idea**: Choose points in a more thoughtful manner

- `np.geomspace` vs. `np.linspace`


In [ ]:
geom_grid = np.geomspace(ce.x_grid.min(), ce.x_grid.max(), num=ce.x_grid.size)

In [ ]:
plt.figure(figsize=(8, 6))
idx = np.arange(0, ce.x_grid.size)
plt.plot(idx, ce.x_grid, 'o', markersize=2., label='np.linspace')
plt.plot(idx, geom_grid, 'o', markersize=2., label='geomspace')
plt.xlabel('index', size=16)
plt.ylabel(r'$x$', size=16)
plt.legend()
plt.title('np.linspace vs np.geomspace');

In [ ]:
ce_geom = CakeEating()
ce_geom.x_grid = geom_grid  # Modifies the grid inplacey

In [ ]:
v_geom = compute_value_function(ce_geom)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x_grid, v, label='np.linspace')
ax.plot(x_grid, interp(geom_grid, v_geom, x_grid), label='np.geomspace')
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
plt.show()

- Fancy: [Adaptive sparse grids](http://johannesbrumm.com/wp-content/uploads/2017/09/Brumm-Scheidegger-2017-ECTA.pdf)

<img src="full_vs_sparse_grid.png" width="400">

<img src="adaptive_sparse_test.png"  width="1000">

*Source: [2]*

**Important note:** There are grid-free methods for doing numerical DP


### Different Approximation Methods

Packages in Python: [interpolation.py](https://github.com/EconForge/interpolation.py), [SciPy](https://docs.scipy.org/doc/scipy/reference/interpolate.html)

- Piecewise linear
- Various types of polynomials ([Chebychev](https://en.wikipedia.org/wiki/Chebyshev_polynomials), [Hermite](https://en.wikipedia.org/wiki/Hermite_polynomials))
- [Splines](https://en.wikipedia.org/wiki/Spline_(mathematics)) (i.e. piecewise polynomial)
- Neural Networks (tools: [TensorFlow](https://github.com/tensorflow/tensorflow), [PyTorch](https://github.com/pytorch/pytorch), [Keras](https://keras.io/))

On Neural Networks: 
- Very popular recently
- [Universal approximation theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem) is "smoke and mirrors"

#### Things to take into consideration:

- Nonexpansive approximation (see [6])
- Shape preservation (see [7])

### Example

Consider the following functional form: $w_{i}\left(y\right)\equiv w\left(y;a_{i},b_{i}\right)=a_{i}y^{b_{i}}$.

(Recall the analytical solution is $v^{*}\left(x\right)=\left(1-\beta^{\frac{1}{\gamma}}\right)^{-\gamma}\frac{x^{1-\gamma}}{1-\gamma}$) 

#### Procedure

Start with $a_{0}$ and $b_{0}$. 

Compute $\hat{T}v_{i+1}\left(x\right)=\max_{c\in\Gamma\left(x\right)}\left\{ u\left(c\right)+\beta w_{i}\left(x-c\right)\right\} $ for all $x$.

Compute $w_{i+1}$ by solving $\min_{\left(a_{i+1},b_{i+1}\right)}N^{-1}\sum_{j=1}^{N}\left(w\left(x_{j};a_{i+1},b_{i+1}\right)-\hat{T}v_{i+1}\left(x_{j}\right)\right)^{2}$.  (We'll use `scipy.optimize.curve_fit` in Python)

Iterate until convergence.


In [ ]:
def w(y, a, b):  # New function
    return a * y ** b


def T(v, ce, a, b):  # Changed here
    """
    The Bellman operator.  Updates the guess of the value function.

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)

    for i, x in enumerate(ce.x_grid):
        # Maximize RHS of Bellman equation at state x
        objective = lambda c, a, b: ce.u(c) + ce.β * w(x - c, a, b)  # Changed here
        v_new[i] = maximize(objective, 1e-10, x, (a, b))[1]
        
    a_new, b_new = curve_fit(w, x_grid, v_new)[0]  # Changed here

    return v_new, a_new, b_new  # Changed here

In [ ]:
def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1
    a, b = 1, 1.  # New variables

    while i < max_iter and error > tol:
        v_new, a, b = T(v, ce, a, b)  # Changed here

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new, a, b  # Changed here

In [ ]:
v_param, a, b = compute_value_function(ce)

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_grid, v_analytical, label='analytical solution')
ax.plot(x_grid, v_param, label='numerical solution')
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
ax.set_title('Comparison between analytical and numerical value functions')
plt.show()

### Different Maximization Routines

- [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)
- [quantecon.optimize](https://quanteconpy.readthedocs.io/en/latest/optimize.html)

In [ ]:
# Define a function to solve the maximization problem
def maximize(g, x0, args):  # Changed here
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    # Previously: result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    result = minimize(objective, x0, bounds=[[1e-10, args[0]]], method='L-BFGS-B')  # Changed here
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum


def T(v, ce):
    """
    The Bellman operator.  Updates the guess of the value function.

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)

    for i, x in enumerate(ce.x_grid):
        # Maximize RHS of Bellman equation at state x
        x0 = x / 10
        v_new[i] = maximize(ce.state_action_value, x0, (x, v))[1]  # Changed here

    return v_new


def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1

    while i < max_iter and error > tol:
        v_new = T(v, ce)

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new

In [ ]:
%%time

v_diff_optimizer = compute_value_function(ce)

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_grid, v, label='original optimizer')
ax.plot(x_grid, v_diff_optimizer, label='new optimizer')
#ax.plot(x_grid, v_analytical, label='analytical')
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
ax.set_title('Comparison between analytical and numerical value functions')
plt.show()

In [ ]:
v_diff_optimizer[0]

In [ ]:
ce.u(x_grid.min()) / (1 - ce.β)

### Reinforcement Learning


"The approximate $|S|$ of
chess, shogi, and Go are $10^{47}$, $10^{71}$, and $10^{171}$, respectively, which are comparable to the
number of atoms in the observable universe ($10^{78}$ ∼ $10^{82}$) and certainly larger than the total
information-storage capacity of humanity (in the order of $10^{20}$ bytes)." (Igami, 2018)

https://deepmind.com/research/case-studies/alphago-the-story-so-far

<img src="fig_8_sutton_barto.png">

*Source: [4]*

#### Exhaustive Search Direction

Expand the Bellman equation:

$$\begin{eqnarray}
v\left(x\right)&=&\max_{c\in\Gamma\left(x\right)}\left\{ u\left(c\right)+\beta v\left(x-c\right)\right\} \\&=&\max_{c_{1}\in\Gamma\left(x\right)}\left\{ u\left(c_{1}\right)+\beta\max_{c_{2}\in\Gamma\left(x\right)}\left\{ u\left(c_{2}\right)+\beta v\left(x-c_{1}-c_{2}\right)\right\} \right\} \\&=&\max_{c_{1}\in\Gamma\left(x\right)}\left\{ u\left(c_{1}\right)+\beta\max_{c_{2}\in\Gamma\left(x\right)}\left\{ u\left(c_{2}\right)+\beta\max_{c_{3}\in\Gamma\left(x\right)}\left\{ u\left(c_{3}\right)+\beta v\left(x-c_{1}-c_{2}-c_{3}\right)\right\} \right\} \right\} 
\end{eqnarray}$$

You can repeat this as many times as you want.

Let $T'v\left(x\right)=\max_{c_{1}\in\Gamma\left(x\right)}\left\{ u\left(c_{1}\right)+\beta\max_{c_{2}\in\Gamma\left(x\right)}\left\{ u\left(c_{2}\right)+\beta\max_{c_{3}\in\Gamma\left(x\right)}\left\{ u\left(c_{3}\right)+\beta v\left(x-c_{1}-c_{2}-c_{3}\right)\right\} \right\} \right\} $.

In [ ]:
class CakeEating:
    def __init__(self,
                 β=0.96,           # discount factor
                 γ=1.5,            # degree of relative risk aversion
                 x_grid_min=1e-3,  # exclude zero for numerical stability
                 x_grid_max=2.5,   # size of cake
                 x_grid_size=120):

        self.β, self.γ = β, γ

        # Set up grid
        self.x_grid = np.linspace(x_grid_min, x_grid_max, x_grid_size)

    # Utility function
    def u(self, c):

        γ = self.γ

        if γ == 1:
            return np.log(c)
        else:
            return (c ** (1 - γ)) / (1 - γ)

    # first derivative of utility function
    def u_prime(self, c):

        return c ** (-self.γ)

    def state_action_value(self, c, x, v_array):
        """
        Right hand side of the Bellman equation given x and c.
        
        """

        u, β = self.u, self.β
        v = lambda x: interp(self.x_grid, v_array, x)

        return u(c[0]) + β * u(c[1]) + β ** 2 * v(x - c[0] - c[1])  # Changed here
    

# Define a function to solve the maximization problem
def maximize(g, x0, args):  # Changed here
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    # Previously: result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    result = minimize(objective, x0, bounds=[[1e-10, args[0]], [1e-10, args[0]]], method='L-BFGS-B')  # Changed here
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum


def T(v, ce):
    """
    The Bellman operator.  Updates the guess of the value function.

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)

    for i, x in enumerate(ce.x_grid):
        # Maximize RHS of Bellman equation at state x
        x0 = np.array([x / 10, x / 10])
        v_new[i] = maximize(ce.state_action_value, x0, (x, v))[1]  # Changed here

    return v_new


def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1

    while i < max_iter and error > tol:
        v_new = T(v, ce)

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new

In [ ]:
%%time

ce = CakeEating()
v_new = compute_value_function(ce)

**Trade-off:** Each iteration is "closer to reality" but more expensive to compute

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_grid, v, label='Iterate on $T$')
ax.plot(x_grid, v_new, label=r"Iterate on $T'$")
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
ax.set_title('Comparison between different methods')
plt.show()

## Parallel Computing and High Performance Computing

**Basic idea:** Use multiple processors at the same time

Most likely, your computer uses a Central Processing Unit (CPU) to run computations. 

**What's a CPU?**

"A central processing unit (CPU), also called a central processor or main processor, is the electronic circuitry within a computer that executes instructions that make up a computer program.The CPU performs basic arithmetic, logic, controlling, and input/output (I/O) operations specified by the instructions in the program." (Wikipedia)

A CPU core is a CPU's processor. Modern CPUs usually have multiple cores.

A thread is a sequence of instructions within a program that can be executed independently of other code. Modern CPU cores generally allow for having multiple threads. This is called hyper-threading. 

My laptop has 2 cores and 4 threads. Modern, consumer-grade CPUs have up to 64 cores and 128 threads (see [here](https://www.cpu-monkey.com/en/cpu-amd_ryzen_threadripper_3990x-977)). 

**Serial vs. parallel computing**

<img src="parallel_1.png" width='600'>
<img src="parallel_2.png" width='600'>

First, stack the objects that need to be computed at a given iteration:

$$\begin{eqnarray}
Tv_{i+1}\left(x_{1}\right)&=&\max_{c\in\Gamma\left(x\right)}\left\{ u\left(c\right)+\beta v_{i}\left(x_{1}-c\right)\right\} \\Tv_{i+1}\left(x_{2}\right)&=&\max_{c\in\Gamma\left(x\right)}\left\{ u\left(c\right)+\beta v_{i}\left(x_{2}-c\right)\right\} \\\vdots&\vdots&\vdots\\Tv_{i+1}\left(x_{N}\right)&=&\max_{c\in\Gamma\left(x\right)}\left\{ u\left(c\right)+\beta v_{i}\left(x_{N}-c\right)\right\} 
\end{eqnarray}$$

$Tv_{i+1}\left(x_{0}\right)$ and $Tv_{i+1}\left(x_{1}\right) $ can be computed **independently** so why not do them in parallel?

#### Is this hard?

It depends.

In [ ]:
# Rewrote the code to work with Numba
def bellman_operator_factory(x_grid, γ, β):
    if γ == 1:
        @njit
        def u(c):
            return np.log(c)    
    else:
        @njit
        def u(c):
            return (c ** (1 - γ)) / (1 - γ)
    
    @njit
    def state_action_value(c, x, v_array):
        """
        Right hand side of the Bellman equation given x and c.
        """
        return u(c) + β * interp(x_grid, v_array, x - c)
    
    
    @njit(parallel=True)  # Equivalent to T = njit(T, parallel=True)
    def T(v):
        """
        The Bellman operator.  Updates the guess of the value function.

        * ce is an instance of CakeEating
        * v is an array representing a guess of the value function

        """
        v_new = np.empty_like(v)

        for i in prange(len(x_grid)):
            # Maximize RHS of Bellman equation at state x
            v_new[i] = scalar_maximization.brent_max(state_action_value, 1e-10, x_grid[i], (x_grid[i], v))[1]

        return v_new
    
    return T


T = bellman_operator_factory(ce.x_grid, ce.γ, ce.β)

In [ ]:
def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1
    T = bellman_operator_factory(ce.x_grid, ce.γ, ce.β)

    while i < max_iter and error > tol:
        v_new = T(v)

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new

In [ ]:
%%time

ce = CakeEating(x_grid_size=100_000)
v_parallel = compute_value_function(ce)

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_grid, v, label='base')
ax.plot(x_grid, interp(ce.x_grid, v_parallel, x_grid), label=r"parallel")
ax.set_ylabel('$V(x)$', fontsize=12)
ax.set_xlabel('$x$', fontsize=12)
ax.legend()
ax.set_title('Comparison between different methods')
plt.show()

**More about parallel computing and HPC**

- [Simon Scheidegger](https://sites.google.com/site/simonscheidegger/home)
- https://github.com/sischei/OSE2019

## Stochastic DP

Consider the following modification of the cake eating problem:

$$\max_{\left\{ c_{t}\right\} _{t=0}^{\infty}}\mathbb{E}\left[\sum_{t=0}^{\infty}\beta^{t}u\left(c_{t}\right)\right]$$

subject to:

$$x_{t+1}=\left(1-\delta_{t+1}\right)\left(x_{t}-c_{t}\right)$$

where $\mathbb{P}\left(\delta_{t}=0\right)=\mathbb{P}\left(\delta_{t}=0.05\right)=0.5$. (i.e. the agent loses 5% of his remaining cake at each period with probability $\frac{1}{2}$).

The associated Bellman equation is:

$$\begin{eqnarray}
v^{*}\left(x\right)&=&\sup_{0\leq c\leq x}u\left(c\right)+\beta\mathbb{E}\left[v^{*}\left(x'\right)\right]\\&=&\sup_{0\leq c\leq x}u\left(c\right)+\beta\sum_{\delta'}\mathbb{\mathbb{P}}\left(\delta'\right)v^{*}\left(x'\right)
\end{eqnarray}$$

The associated Bellman operator satisfies:

$$\left(Tv\right)\left(x\right)=\sup_{0\leq c\leq x}u\left(c\right)+\beta\sum_{\delta'}\mathbb{\mathbb{P}}\left(\delta'\right)v\left(x'\right)$$

In [ ]:
class CakeEating:
    def __init__(self,
                 β=0.96,           # discount factor
                 γ=1.5,            # degree of relative risk aversion
                 π=0.5,            # P(δ=0)  -- changed here
                 x_grid_min=1e-3,  # exclude zero for numerical stability
                 x_grid_max=2.5,   # size of cake
                 x_grid_size=120):

        self.β, self.γ = β, γ
        self.δ_vals = np.array([0., 0.05])  # Changed here
        self.π = π  # Changed here

        # Set up grid
        self.x_grid = np.linspace(x_grid_min, x_grid_max, x_grid_size)

    # Utility function
    def u(self, c):

        γ = self.γ

        if γ == 1:
            return np.log(c)
        else:
            return (c ** (1 - γ)) / (1 - γ)

    # first derivative of utility function
    def u_prime(self, c):

        return c ** (-self.γ)

    def state_action_value(self, c, x, v_array):  # Changed this function
        """
        Right hand side of the Bellman equation given x and c.
        
        """

        u, β = self.u, self.β
        δ_vals = self.δ_vals
        π = self.π
        
        v = lambda x: interp(self.x_grid, v_array, x)
        Ev = π * v((1 - δ_vals[0]) * (x - c)) + (1 - π) * v((1 - δ_vals[1]) * (x - c))

        return u(c) + β * Ev
    

# Define a function to solve the maximization problem
def maximize(g, a, b, args):
    """
    Maximize the function g over the interval [a, b].

    We use the fact that the maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximal value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum


def T(v, ce):
    """
    The Bellman operator.  Updates the guess of the value function.

    * ce is an instance of CakeEating
    * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)

    for i, x in enumerate(ce.x_grid):
        # Maximize RHS of Bellman equation at state x
        v_new[i] = maximize(ce.state_action_value, 1e-10, x, (x, v))[1]  # v_new[i] = v'(x_i)

    return v_new

In [ ]:
def compute_value_function(ce,
                           tol=1e-4,
                           max_iter=1000,
                           verbose=True,
                           print_skip=25):

    # Set up loop
    v = np.zeros(len(ce.x_grid)) # Initial guess
    i = 0
    error = tol + 1

    while i < max_iter and error > tol:  # `max_iter` is used to ensure termination of algorithm
        v_new = T(v, ce)

        error = np.max(np.abs(v - v_new))
        i += 1

        if verbose and i % print_skip == 0:
            print(f"Error at iteration {i} is {error}.")

        v = v_new

    if i == max_iter:
        print("Failed to converge!")

    if verbose and i < max_iter:
        print(f"\nConverged in {i} iterations.")

    return v_new

In [ ]:
ce = CakeEating()

In [ ]:
v_stoch = compute_value_function(ce)

In [ ]:
plt.plot(x_grid, v, label='original problem')
plt.plot(x_grid, v_stoch, label='modified problem')
plt.legend();

## Q-Learning

**Basic idea:** Sample the expectation instead of computing it exactly

Let: 

$$Q\left(x,c\right)=u\left(c\right)+\beta\mathbb{E}\left[v^{*}\left(x'\right)\right]$$

so that:

$$v^{*}\left(x\right)=\sup_{0\leq c\leq x}Q\left(x,c\right)$$.

We'll focus on computing $Q\left(x,c\right)$ instead of $v^{*}\left(x\right)$.

**Algorithm:**

First, generate a sample $x_{t+1}$ given $c_{t}$ and $x_{t}$ 

Then, compute:

$$\underbrace{Q^{new}\left(x_{t},c_{t}\right)}_{\mathrm{new\,value}}\leftarrow\underbrace{Q\left(x_{t},c_{t}\right)}_{\mathrm{old\,value}}+\underbrace{\alpha}_{\mathrm{learning\,rate}}\underbrace{\left(u\left(c_{t}\right)+\beta\max_{c}Q\left(x_{t+1},c\right)-Q\left(x_{t},c_{t}\right)\right)}_{\mathrm{temporal\,difference}}$$

When $\alpha=1$, notice that this becomes:

$$Q^{new}\left(x_{t},c_{t}\right)\leftarrow u\left(c_{t}\right)+\beta\max_{c}Q\left(x_{t+1},c\right)$$

Notice that the update is stochastic. 

To get it to converge, at each iteration do:

$$\alpha\leftarrow0.999\alpha$$ 

(Note: This is a little bit different than the algorithm in [4] but captures the essential idea)

In [ ]:
def bellman_operator_factory(x_grid, c_grid, γ, β, δ_vals):
    x_grid_min = x_grid.min()
    grid = UCGrid((x_grid.min(), x_grid.max(), x_grid.size), (c_grid.min(), c_grid.max(), c_grid.size))
    eval_points = np.empty((c_grid.size, 2))
    eval_points[:, 1] = c_grid
    
    if γ == 1:
        @njit
        def u(c):
            return np.log(c)    
    else:
        @njit
        def u(c):
            return (c ** (1 - γ)) / (1 - γ)
    
    @njit
    def T(Q, v, α):
        Q_new = Q.copy()

        for x_i, x in enumerate(x_grid):
            for c_i, c in enumerate(c_grid):
                δ_i = np.random.randint(2)
                xp = (1 - δ_vals[δ_i]) * (x - c)
                v_xp = interp(x_grid, v, xp)
                
                if xp > 0.:  # Only update well-defined state action pairs
                    temp_diff = u(c) + β * v_xp - Q[x_i, c_i]
                    Q_new[x_i, c_i] = Q[x_i, c_i] + α * (temp_diff)

        return Q_new
    
    return T

# Multiply min x by 0.99 for the (0, 0) state action pair to be well defined
c_grid = np.linspace(x_grid.min() * 0.99, x_grid.max(), x_grid.size * 4)  
T = bellman_operator_factory(ce.x_grid, c_grid, ce.γ, ce.β, ce.δ_vals)

In [ ]:
max_itr = 50_000

Δs = []
Q = np.ones((ce.x_grid.size, c_grid.size)) * -100_000  # Trick to ignore state action pairs that are not well defined

itr = 1
α = 1.

while True:
    v = Q.max(axis=1)
    Q_new = T(Q, v, α)
    Δs.append(np.max(np.abs(Q_new - Q)))
    Q = Q_new
    
    if itr % 1000 == 0:
        print(f"Error at iteration {itr} is {Δs[itr-1]}.")
    
    if Δs[itr-1] < 1e-7:
        break
        
    if itr == max_itr:
        break
    
    itr += 1
    α = α * 0.999  # Decay learning rate
    
    
if itr == max_itr:
    print("Failed to converge!")

if itr < max_itr:
    print(f"\nConverged in {itr} iterations.")

In [ ]:
min_itr = 100
max_itr = 300

plt.plot(np.arange(min_itr, max_itr+1), Δs[min_itr-1:max_itr])
plt.ylabel(r'$\left\Vert Q^{new}-Q\right\Vert _{\infty}$')
plt.xlabel('Iteration');

In [ ]:
plt.plot(v_stoch, label='DP')
plt.plot(Q.max(axis=1), label='Q-learning')
plt.legend();

## References

[1] John Stachurski, 2009. "Economic Dynamics: Theory and Computation," MIT Press Books  
[2] Johannes Brumm & Simon Scheidegger, 2017. "Using Adaptive Sparse Grids to Solve High‐Dimensional Dynamic Models," Econometrica  
[3] Artificial Intelligence as Structural Estimation: Economic Interpretations of Deep Blue, Bonanza, and AlphaGo, Mitsuru Igami  
[4] Richard S. Sutton and Andrew G. Barto. 2018. Reinforcement Learning: An Introduction. A Bradford Book, Cambridge, MA, USA.  
[5] [Daisuke Oyama's](http://www.oyama.e.u-tokyo.ac.jp/) excellent lecture notes on Dynamic Programming  
[6] John Stachurski, 2008. "Continuous State Dynamic Programming via Nonexpansive Approximation," Computational Economics  
[7] Yongyang Cai & Kenneth Judd, 2013. "Shape-preserving dynamic programming," Mathematical Methods of Operations Research  
[8] Cai, Yongyang & Judd, Kenneth. (2014). Advances in Numerical Dynamic Programming and New Applications. Handbook of Computational Economics.